In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.optimize import curve_fit
from scipy.stats import linregress

In [ ]:
# Capstone guidelines: https://bcourses.berkeley.edu/courses/1532916/pages/capstone-project-2?module_item_id=17001677

In [5]:
# load data
def file_name_voltage(number):
    return f"resources/data/{number}_voltage.csv"

def file_name_B_field(number):
    return f"resources/data/{number}_B_field.csv"

data0_1_table = np.genfromtxt(file_name_voltage(0.1), delimiter=',', skip_header=1)
data0_1= {
    "time": data0_1_table[:, 0],        # time, s
    "voltage": data0_1_table[:, 1],     # voltage, V
}

data0_2_table = np.genfromtxt(file_name_voltage(0.2), delimiter=',', skip_header=1)
data0_2= {
    "time": data0_2_table[:, 0],        # time, s
    "voltage": data0_2_table[:, 1],     # voltage, V
}

data1_1_B_table = np.genfromtxt(file_name_B_field(1.1), delimiter=',', skip_header=1)
data1_1_B= {
    "time": data1_1_B_table[:,0], # time, s
    "Bx": data1_1_B_table[:,1],   # Bx (µT)
    "By": data1_1_B_table[:,2],   # By (µT)
    "Bz": data1_1_B_table[:,3],   # Bz (µT)
}

data1_1_v_table = np.genfromtxt(file_name_voltage(1.1), delimiter=',', skip_header=1)
data1_1_v= {
    "time": data1_1_v_table[:, 0],        # time, s
    "voltage": data1_1_v_table[:, 1],     # voltage, V
}

data2_1_B_table = np.genfromtxt(file_name_B_field(2.1), delimiter=',', skip_header=1)
data2_1_B = {
    "time": data2_1_B_table[:, 0],  # time, s
    "Bx": data2_1_B_table[:, 1],    # Bx (µT)
    "By": data2_1_B_table[:, 2],    # By (µT)
    "Bz": data2_1_B_table[:, 3],    # Bz (µT)
}

data2_1_v_table = np.genfromtxt(file_name_voltage(2.1), delimiter=',', skip_header=1)
data2_1_v = {
    "time": data2_1_v_table[:, 0],        # time, s
    "voltage": data2_1_v_table[:, 1],     # voltage, V
}

data2_2_B_table = np.genfromtxt(file_name_B_field(2.2), delimiter=',', skip_header=1)
data2_2_B = {
    "time": data2_2_B_table[:, 0],  # time, s
    "Bx": data2_2_B_table[:, 1],    # Bx (µT)
    "By": data2_2_B_table[:, 2],    # By (µT)
    "Bz": data2_2_B_table[:, 3],    # Bz (µT)
}

data2_2_v_table = np.genfromtxt(file_name_voltage(2.2), delimiter=',', skip_header=1)
data2_2_v = {
    "time": data2_2_v_table[:, 0],        # time, s
    "voltage": data2_2_v_table[:, 1],     # voltage, V
}

data2_3_B_table = np.genfromtxt(file_name_B_field(2.3), delimiter=',', skip_header=1)
data2_3_B = {
    "time": data2_3_B_table[:, 0],  # time, s
    "Bx": data2_3_B_table[:, 1],    # Bx (µT)
    "By": data2_3_B_table[:, 2],    # By (µT)
    "Bz": data2_3_B_table[:, 3],    # Bz (µT)
}

data2_3_v_table = np.genfromtxt(file_name_voltage(2.3), delimiter=',', skip_header=1)
data2_3_v = {
    "time": data2_3_v_table[:, 0],        # time, s
    "voltage": data2_3_v_table[:, 1],     # voltage, V
}

data2_4_B_table = np.genfromtxt(file_name_B_field(2.4), delimiter=',', skip_header=1)
data2_4_B = {
    "time": data2_4_B_table[:, 0],  # time, s
    "Bx": data2_4_B_table[:, 1],    # Bx (µT)
    "By": data2_4_B_table[:, 2],    # By (µT)
    "Bz": data2_4_B_table[:, 3],    # Bz (µT)
}

data2_4_v_table = np.genfromtxt(file_name_voltage(2.4), delimiter=',', skip_header=1)
data2_4_v = {
    "time": data2_4_v_table[:, 0],        # time, s
    "voltage": data2_4_v_table[:, 1],     # voltage, V
}

def get_data_slice(data, begin_second, end_second):
    if begin_second < 1/200: begin_second = 1/200
    return {key: value[int(begin_second*200-1):int(end_second*200-1)] for key, value in data.items()}

In [2]:
# part 1: analyze data for original solonoid, showing that the physics works for each variable change

In [ ]:
'''
Trial 1
110 grams plus magnet moving on spring (magnet = .013 g)
2 trials done - capstone test in Theo cloud (0.1, 0.2)
only induced voltage recorded
just using 1 magnet
P1T1.jpg
'''
def part_1_t_1():
    return 0

In [ ]:
'''
Trial 2
110 grams plus magnet moving on spring (magnet = .013 g)
now figuring out best way to measure displacement - putting Iolab over magnet and measuring magnetometer which is proportional to displacement
Displacement in will cloud, induced voltage in Theo
Both saved as: Ex.1.1 Capstone
P1T2.jpg
'''
def part_1_t_2():
    return 0

In [ ]:
'''
Trial 3
2 solenoids
110 grams plus magnet moving on spring (magnet = .013 g)
Displacement in will cloud, induced voltage in Theo
Both saved as: Ex. 2.1 Capstone
P1T3.jpg
'''
def part_1_t_3():
    return 0

In [ ]:
'''
Trial 4
same as trial 3, but moved the iolab up so no longer starts in solonoid
saved as Ex. 2.2 Capstone
P1T4_5_6.jpg
'''
def part_1_t_4():
    return 0

In [ ]:
'''
Trial 5
Same as trial 4 but 230 grams plus magnet
Both saved as: Ex. 2.3 Capstone
P1T4_5_6.jpg
'''
def part_1_t_5():
    return 0

In [ ]:
'''
Trial 6
same as trial 5 but magnet strength = more (more magnets)
Using 2 magnets
both saved as Ex. 2.4 Capstone
P1T4_5_6.jpg
'''
def part_1_t_6():
    return 0

In [ ]:
#: part 2 - create 2 solenoids, different radii - show emf is proportional to radius squared
# one using gatorade bottle, other using mints jar

In [ ]:
def part_2():
    return 0

In [ ]:
# part 3: add a iron core (some ferromagnetic material) to the bottom half of the solenoid, increase self-inductance
# emf = L * di/dt

In [ ]:
def part_3():
    return 0